In [1]:
from morphocycle.models.coatnet import create_model

In [2]:
model = create_model()

In [3]:
model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [4]:
def calculate_loss(self, inputs, labels):
    logits = self.forward(inputs)
    loss = self.criterion(logits, labels.long())
    y_prob = torch.softmax(logits, dim=1)[:, 1]
    y_hat = torch.argmax(y_prob, dim=1)
    return loss, y_prob, y_hat, logits


In [5]:
import numpy as np
import torch
from torch.utils.data import Dataset
from pathlib import Path
import tifffile as tfl
import torchvision.transforms as T

class CellCycleData(Dataset):
    """
    Dataset class for the single cell dataset
    """

    def __init__(
        self,
        img_dir=None,
    ):
        # Set all input args as attributes
        self.__dict__.update(locals())
        self.img_dir = Path(img_dir)

        # Get all the image files
        img_files = list(self.img_dir.glob("**/*/*/*.tif"))
        self.img_files = [i for i in img_files if "NotKnown" not in str(i)]

        # Get all the labels
        labels = [str(x.parent.name) for x in self.img_files]
        self.label_dict = {"G1": 0,
                           "G1-S": 1,
                           "S": 2,
                           "S-G2": 3,
                           "G2": 4,
                           "G2-M": 5,
                           "M": 6,
                           "MorG1": 7}
        self.labels = [self.label_dict[x] for x in labels]
        self.track_ids = [str(x.parent.parent.name) for x in self.img_files]
        self.slide_ids = [str(x.parent.parent.parent.name) for x in self.img_files]

    def __len__(self):
        return len(self.img_files)

    def __getitem__(self, idx):
        img_path = self.img_files[idx]
        img = tfl.imread(img_path)
#         v_min, v_max = img.min(), img.max()
#         new_min, new_max = 0, 255
#         img = (img - v_min) / (v_max - v_min) * (new_max - new_min) + new_min
        img = img/(img.max() + 1e-5)
        transform = T.Compose([T.ToTensor(),
                               T.RandomHorizontalFlip(p=0.5),
                               T.RandomVerticalFlip(p=0.5),
                               T.RandomRotation(degrees=90),
                               T.RandomPerspective(distortion_scale=0.5, p=0.5),
                               T.Resize((64, 64))
                               ])

        img = transform(img)
        img = img.expand(3, *img.shape[1:]).type(torch.FloatTensor)
        label = torch.tensor(self.labels[idx])
        track_id = self.track_ids[idx]
        slide_id = self.slide_ids[idx]


        return img, label, track_id, slide_id

In [6]:
dset = CellCycleData("/media/mvries/Derek_Jeeters/PCNA_cell_cycle_marker/data_analysis/CycleData")

In [68]:
dset[4][0].mean()

tensor(0.0381)

In [8]:

from torch.utils.data import DataLoader
dload = DataLoader(dset, batch_size=256, shuffle=True)

In [33]:
from tqdm import tqdm

all_results = []

for i, d in tqdm(enumerate(dload)):
#     print(d[0].isnan().any())

#     logits = model(d[0])
#     y_prob = torch.softmax(logits, dim=1)
#     y_hat = torch.argmax(y_prob, dim=1)
    all_results.append({
#             "logits": logits,
#             "Y_prob": y_prob,
#             "Y_hat": y_hat,
            "label": d[1],
        })

603it [06:29,  1.55it/s]


KeyboardInterrupt: 

In [29]:
all_results[0]['label'].shape

torch.Size([256])

In [12]:
max_probs = torch.stack([x["Y_hat"] for x in all_results])

In [20]:
max_probs

tensor([[6, 4, 3,  ..., 1, 3, 6],
        [6, 6, 6,  ..., 6, 6, 6],
        [6, 6, 6,  ..., 6, 6, 6],
        ...,
        [6, 6, 6,  ..., 6, 6, 6],
        [6, 6, 6,  ..., 6, 6, 6],
        [6, 6, 6,  ..., 6, 6, 6]])

In [74]:
y_hat = torch.argmax(y_prob, dim=1)

In [19]:
y_hat.shape

torch.Size([256])

In [31]:
torch.cat((torch.ones((256)), torch.ones((256)), torch.zeros((120))), dim=0)

tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 

In [77]:
import torch.nn as nn

criterion = nn.CrossEntropyLoss()

In [78]:
criterion(logits, d[1].double().long())

tensor(2.1008, grad_fn=<NllLossBackward0>)

In [81]:
logits.mean()

tensor(-0.0108, grad_fn=<MeanBackward0>)